# nuScenes devkit tutorial

Welcome to the nuScenes tutorial. Make sure you have the nuScenes DB schema available.

This demo assumes the database itself is available at `/data/nuscenes`

In [ ]:
# Let's start by initializing the database
%matplotlib inline
from nuscenes.nuscenes import NuScenes

nusc = NuScenes(version='v0.1', dataroot='/data/nuscenes', verbose=True)

## NuScenes Basics

In [ ]:
# The NuScenes class holds several tables. Each table is a list of records, and each record is a dictionary. 
# For example the first record of the category table is stored at
nusc.category[0]

In [ ]:
# The category table is simple: it just holds fields `name` and `description`. In addition it also has a `token` 
# field, which is a unique record identifier. Since the record is a dictionary, the token can be accessed like so:
cat_token = nusc.category[0]['token']

# If you know the token for any record in the DB you can retrieve the record by doing
nusc.get('category', cat_token)

In [ ]:
# As you notice, we have now recovered the same record!

In [ ]:
# OK, that was pretty easy. Let's try something harder. Let's look at the sample_annotation table.
nusc.sample_annotation[0]

In [ ]:
# As you can see this also has a `token` field (they all do). In addition, it has several fields of the format [
# a-z]*_token, e.g. instance_token. These are foreign keys in database speak, meaning they point to another table. 
# Using nusc.get() we can grab any of these in constant time. For example, let's look at the visibility record.
nusc.get('visibility', nusc.sample_annotation[0]['visibility_token'])

In [ ]:
# The visibility records informs of how much of the object was visible when it was annotated.

# Let's also grab the instance_token
one_instance = nusc.get('instance', nusc.sample_annotation[0]['instance_token'])
one_instance

In [ ]:
# This points to the instance table. This table enumerate the object _instances_ we have encountered in each 
# scene. This way we can connect all annotation of a particular object.

# If you look carefully at the README tables, you will see that the sample_annotation table points to the object table, 
# but the object table doesn't list all annotations that point to it. 

# So how can we recover all sample_annotations for a particular object? There are two ways:

# (1). Use nusc.field2token(). Let's try it:
ann_tokens = nusc.field2token('sample_annotation', 'instance_token', one_instance['token'])

# This returns a list of all sample_annotation records with the `instance_token` == one_instance['token'].
# Let's store these in a set for now
ann_tokens_field2token = set(ann_tokens)

ann_tokens_field2token

In [ ]:
# The nusc.field2token() method is generic and can be used in any similar situation.

# (2) For certain situation, we provide some reverse indices in the tables themselves. This is one such example. 
# The instance record has a field `first_annotation_token` which points to the first annotation in time of this 
# instance. Recovering this record is easy.
ann_record = nusc.get('sample_annotation', one_instance['first_annotation_token'])
ann_record

In [ ]:
# Now we can traverse all annotations of this instance using the "next" field. Let's try it. 
ann_tokens_traverse = set()
ann_tokens_traverse.add(ann_record['token'])
while not ann_record['next'] == "":
    ann_record = nusc.get('sample_annotation', ann_record['next'])
    ann_tokens_traverse.add(ann_record['token'])

# Finally, let's assert that we recovered the same ann_records as we did using nusc.field2token:
assert ann_tokens_traverse == ann_tokens_field2token

## Reverse indexing and short-cuts

The nuScenes tables are normalized, meaning that each piece of information is only given once.
For example, there is one map record for each log record. Looking at the schema you will notice that the `map` table has a `log_token` field, but that the `log` table does not have a corresponding `map_token` field. But there are plenty of situations where you have a log, and want to find the corresponding map! So what to do? You can always use the `nusc.field2token()` method, but that is slow and inconvenient. We therefore add reverse mappings for some common situations including this one.

Further, there are situations where one needs to go through several tables to get a certain piece of information. 
Consider, for example, the category name (e.g. `human.pedestrian`) of a `sample_annotation`. The `sample_annotation` table doesn't hold this information since the category is an instance level constant. Instead the `sample_annotation` table points to a record in the `instance` table. This, in turn, points to a record in the `category` table, where finally the `name` fields stores the required information. Since it is quite common to want to know the category name of an annotation, we add a `category_name` field to the `sample_annotation` table during initialization of the NuScenes class.

In this section we list the short-cuts and reverse indices that are added to the `NuScenes` class during initialization. These are all created during in the `NuScenes.__make_reverse_index__()` method.

### Shortcuts

In [ ]:
# The sample_annotation table has a "category_name" shortcut:
catname = nusc.sample_annotation[0]['category_name']

ann_rec = nusc.sample_annotation[0]
inst_rec = nusc.get('instance', ann_rec['instance_token'])
cat_rec = nusc.get('category', inst_rec['category_token'])
print(catname == cat_rec['name'])

In [ ]:
# The sample_data table has "channel" and "sensor_modality" shortcuts:
channel = nusc.sample_data[0]['channel']

sd_rec = nusc.sample_data[0]
cs_record = nusc.get('calibrated_sensor', sd_rec['calibrated_sensor_token'])
sensor_record = nusc.get('sensor', cs_record['sensor_token'])
print(channel == sensor_record['channel'])

### Reverse indices
We add two reverse indices by default.
* A `map_token` field is added to the `log` records.
* The `sample` records have shortcuts to all `sample_annotations` for that record as well as `sample_data` key-frames. Confer `nusc.list_sample()` method in next section for more details on this.

## Data Visualizations

We provide a set of list and rendering methods. These are meant both as convenience methods during development and as tutorials for building your own. They are implemented in the NuScenesExplorer class, with shortcuts through the NuScenes class itself.

### List methods
There are three list methods availble to look at certain aspects of the DB.

In [ ]:
# list_categories() lists all categories, counts and statistics of width/length/height in meters and aspect ratio.
nusc.list_categories()

In [ ]:
# list_attributes() lists all attributes and counts.
nusc.list_attributes()

In [ ]:
# list_scenes() lists all scenes in the DB.
nusc.list_scenes()

In [ ]:
# list_sample() lists all related sample_data keyframes and annotation associated with a sample
nusc.list_sample(nusc.sample[0]['token'])

### Render

In [ ]:
# First, let's plot a lidar point cloud in an image.
# Lidar allows us to accurate map the surroundings in 3D.
my_sample = nusc.sample[1300]
nusc.render_pointcloud_in_image(my_sample['token'], pointsensor_channel='LIDAR_TOP')

In [ ]:
# Second, let's plot the radar point cloud for the same image.
# Radar is less dense than lidar, but has a much larger range.
nusc.render_pointcloud_in_image(my_sample['token'], pointsensor_channel='RADAR_FRONT')

In [ ]:
# We can also plot all annotations across all sample data for that sample.
# Note how for radar we also plot the velocity vectors of moving objects.
# Some velocity vectors are outliers, which can be filtered using the settings in RadarPointCloud.from_file().
my_sample = nusc.sample[2958]
nusc.render_sample(my_sample['token'])

In [ ]:
# Or if we only want to render a particular sensor, we can specify that.
nusc.render_sample_data(my_sample['data']['CAM_FRONT'])

In [ ]:
# Additionally we can aggregate the point clouds from multiple sweeps to get a denser point cloud.
nusc.render_sample_data(my_sample['data']['LIDAR_TOP'], nsweeps=5)
nusc.render_sample_data(my_sample['data']['RADAR_FRONT_RIGHT'], nsweeps=5)

In [ ]:
# We can even render a specific annotation.
nusc.render_annotation(my_sample['anns'][22])

In [ ]:
# Finally, we can render a full scene as a video. There are two options here. 
# (1) nusc.render_scene_channel() renders the video for a particular channel. (HIT ESC to exit)
# (2) nusc.render_scene() renders the video for all camera channels.
# NOTE: These methods use OpenCV for rendering, which doesn't always play nice with IPython Notebooks. 
# If you experience any issues please run these lines from the command line. 

# Let's grab scene 0043, it is nice and dense.
my_scene_token = nusc.field2token('scene', 'name', 'scene-0043')[0]
nusc.render_scene_channel(my_scene_token, 'CAM_FRONT')

In [ ]:
# There is also a method nusc.render_scene() which renders the video for all camera channels. 
# This requires a high-res monitor, and is also best run outside this notebook.
nusc.render_scene(my_scene_token)

In [ ]:
# Finally, let us visualize all scenes on the map for a particular location.
nusc.render_egoposes_on_map(log_location='singapore-onenorth')